In [1]:
using Luxor
using Plots
using IterTools
using DataFrames
using OpenStreetMapX
using SparseArrays
using Random

In [2]:
include("../graph.jl")

create_sparse_index (generic function with 1 method)

In [3]:
city = "Warszawa"
admin_level = "6"
search_area = 500
attr = :education
wilderness_distance = 300
shape = "rectangle"
calculate_percent = true
num_of_points = 100
distance_sectors = 200.0
scrape_config = "../poi_config_test.csv"

"../poi_config_test.csv"

In [4]:
dir = "../data"

"../data"

In [5]:
parsed = OpenStreetMapX.parseOSM(string(dir,"/","Warszawa.osm"))

OpenStreetMapX.OSMData(Dict{Int64, LLA}(2517817810 => LLA(52.2918464, 20.9290118, 0.0), 9892161289 => LLA(52.2597255, 20.8778026, 0.0), 10262401922 => LLA(52.2588687, 21.0303653, 0.0), 9261708853 => LLA(52.2130234, 21.0911806, 0.0), 1817529358 => LLA(52.0990631, 21.0553577, 0.0), 10775441465 => LLA(52.1798866, 20.9017401, 0.0), 3296902170 => LLA(52.2906129, 20.928142, 0.0), 9289118246 => LLA(52.2422821, 20.8552067, 0.0), 1423387313 => LLA(52.3536716, 20.9101791, 0.0), 8387783390 => LLA(52.2426216, 21.1048414, 0.0)…), Way[Way(4307329, [2448759046, 7093785352, 2452307268, 1439574696], Dict("name:etymology:wikidata" => "Q5441838", "surface" => "asphalt", "name" => "Rondo Feliksa Stamma", "sidewalk:right" => "separate", "wikidata" => "Q113528575", "lit" => "yes", "highway" => "tertiary", "cycleway:both" => "no", "junction" => "roundabout", "sidewalk:left" => "no"…)), Way(4307330, [26063923, 26063924, 9900672046, 7093724913, 26063925, 4770319996, 7554494461, 9442368604, 11303130233, 1130323

In [6]:
ways = parsed.ways

938116-element Vector{Way}:
 Way(4307329, [2448759046, 7093785352, 2452307268, 1439574696], Dict("name:etymology:wikidata" => "Q5441838", "surface" => "asphalt", "name" => "Rondo Feliksa Stamma", "sidewalk:right" => "separate", "wikidata" => "Q113528575", "lit" => "yes", "highway" => "tertiary", "cycleway:both" => "no", "junction" => "roundabout", "sidewalk:left" => "no"…))
 Way(4307330, [26063923, 26063924, 9900672046, 7093724913, 26063925, 4770319996, 7554494461, 9442368604, 11303130233, 11303232206, 9220363525, 9982770288, 11687239928, 7093724922, 1439574729], Dict("lit" => "yes", "name" => "Bokserska", "source:maxspeed" => "PL:urban", "highway" => "tertiary", "lanes" => "2", "maxspeed" => "50", "surface" => "asphalt", "wikidata" => "Q106807412"))
 Way(4308966, [3387797238, 9252948452], Dict("cycleway:left" => "no", "cycleway:right" => "separate", "lit" => "yes", "name" => "Postępu", "highway" => "tertiary", "lanes" => "2", "sidewalk" => "separate", "maxspeed" => "50", "surface" => 

In [7]:
road_types = ["motorway", "trunk", "primary", "secondary", 
            "tertiary", "residential", "service", "living_street", 
            "motorway_link", "trunk_link", "primary_link", "secondary_link", 
            "tertiary_link"]  

13-element Vector{String}:
 "motorway"
 "trunk"
 "primary"
 "secondary"
 "tertiary"
 "residential"
 "service"
 "living_street"
 "motorway_link"
 "trunk_link"
 "primary_link"
 "secondary_link"
 "tertiary_link"

In [8]:
ws = filter_ways(ways,road_types)

123358-element Vector{Way}:
 Way(4307329, [2448759046, 7093785352, 2452307268, 1439574696], Dict("name:etymology:wikidata" => "Q5441838", "surface" => "asphalt", "name" => "Rondo Feliksa Stamma", "sidewalk:right" => "separate", "wikidata" => "Q113528575", "lit" => "yes", "highway" => "tertiary", "cycleway:both" => "no", "junction" => "roundabout", "sidewalk:left" => "no"…))
 Way(4307330, [26063923, 26063924, 9900672046, 7093724913, 26063925, 4770319996, 7554494461, 9442368604, 11303130233, 11303232206, 9220363525, 9982770288, 11687239928, 7093724922, 1439574729], Dict("lit" => "yes", "name" => "Bokserska", "source:maxspeed" => "PL:urban", "highway" => "tertiary", "lanes" => "2", "maxspeed" => "50", "surface" => "asphalt", "wikidata" => "Q106807412"))
 Way(4308966, [3387797238, 9252948452], Dict("cycleway:left" => "no", "cycleway:right" => "separate", "lit" => "yes", "name" => "Postępu", "highway" => "tertiary", "lanes" => "2", "sidewalk" => "separate", "maxspeed" => "50", "surface" => 

In [9]:
ways_intersections, intersections, road_tags = find_intersections(ws)

(Dict(451965551 => [4978125632, 1181144372], 184475036 => [9265228166, 1949648310], 29448268 => [1495256494, 8213623939, 1495256391], 1147662891 => [10678694819, 10678694818], 109767263 => [1255665807, 1255665867], 499336257 => [785374731, 278765937, 8857026633], 726568927 => [1168754351, 10160234349], 777336914 => [1075952943, 9359581085], 205347889 => [4808482445, 9461423020, 4808482439, 38427866], 964325027 => [8919803371, 8919803372]…), Set([255152525, 4881584092, 5568665390, 9104166618, 11990641011, 324549034, 11215827240, 11994555154, 6832323964, 8893498647  …  9839791340, 1586321166, 7077020185, 1645442346, 10727938096, 10875804121, 3404809274, 11990640786, 11339122480, 10693459631]), Dict(451965551 => Dict("oneway" => "yes", "unsigned_ref" => "5533W", "lanes" => "2", "surface" => "asphalt", "name" => "11 Listopada", "wikipedia" => "pl:Ulica 11 Listopada w Warszawie", "wikidata" => "Q9364834", "lit" => "yes", "highway" => "tertiary", "sidewalk" => "separate"…), 184475036 => Dict

In [10]:
intersections

Set{Int64} with 173818 elements:
  255152525
  4881584092
  5568665390
  9104166618
  11990641011
  324549034
  11215827240
  11994555154
  6832323964
  8893498647
  286630125
  11574682106
  10101862730
  7170513109
  10900387175
  9710077826
  89135815
  5053328354
  9707450528
  ⋮ 

In [11]:
ways_intersections

Dict{Int64, Vector{Int64}} with 123358 entries:
  451965551  => [4978125632, 1181144372]
  184475036  => [9265228166, 1949648310]
  29448268   => [1495256494, 8213623939, 1495256391]
  1147662891 => [10678694819, 10678694818]
  109767263  => [1255665807, 1255665867]
  499336257  => [785374731, 278765937, 8857026633]
  726568927  => [1168754351, 10160234349]
  777336914  => [1075952943, 9359581085]
  205347889  => [4808482445, 9461423020, 4808482439, 38427866]
  964325027  => [8919803371, 8919803372]
  23555908   => [255080040, 7304651612]
  429060793  => [473079996, 4281827250]
  1046867215 => [1075952567, 9626711876]
  1141748128 => [10634627995, 10634627993]
  1163069403 => [731335083, 10816665144]
  1175139604 => [10919800637, 10919800636]
  5080037    => [3722925559, 3722842533, 5140907021, 5140906020, 31158709]
  1091085670 => [9993538156, 9993538153]
  1131779428 => [10552920084, 10552920065]
  ⋮          => ⋮

In [12]:
edges = ways_to_edges(ways_intersections,road_tags,parsed)

357382-element Vector{Any}:
 Edge(1, 4978125632, 1181144372, LLA(52.2566118, 21.0331274, 0.0), LLA(52.2566598, 21.0331933, 0.0), 451965551, "tertiary")
 Edge(2, 1949648310, 9265228166, LLA(52.2159197, 20.979194, 0.0), LLA(52.2154663, 20.9783547, 0.0), 184475036, "service")
 Edge(3, 9265228166, 1949648310, LLA(52.2154663, 20.9783547, 0.0), LLA(52.2159197, 20.979194, 0.0), 184475036, "service")
 Edge(4, 8213623939, 1495256494, LLA(52.2757071, 21.0658142, 0.0), LLA(52.2767636, 21.0666679, 0.0), 29448268, "service")
 Edge(5, 1495256494, 8213623939, LLA(52.2767636, 21.0666679, 0.0), LLA(52.2757071, 21.0658142, 0.0), 29448268, "service")
 Edge(6, 1495256391, 8213623939, LLA(52.2756593, 21.0657277, 0.0), LLA(52.2757071, 21.0658142, 0.0), 29448268, "service")
 Edge(7, 8213623939, 1495256391, LLA(52.2757071, 21.0658142, 0.0), LLA(52.2756593, 21.0657277, 0.0), 29448268, "service")
 Edge(8, 10678694818, 10678694819, LLA(52.1684916, 20.9306972, 0.0), LLA(52.1685319, 20.9307433, 0.0), 1147662891, "

In [13]:
df = edges_to_df(edges)

Row,id,from,to,from_LLA,to_LLA,way,type
,Int64,Int64,Int64,LLA,LLA,Int64,String
1,1,4978125632,1181144372,"LLA(52.2566, 21.0331, 0.0)","LLA(52.2567, 21.0332, 0.0)",451965551,tertiary
2,2,1949648310,9265228166,"LLA(52.2159, 20.9792, 0.0)","LLA(52.2155, 20.9784, 0.0)",184475036,service
3,3,9265228166,1949648310,"LLA(52.2155, 20.9784, 0.0)","LLA(52.2159, 20.9792, 0.0)",184475036,service
4,4,8213623939,1495256494,"LLA(52.2757, 21.0658, 0.0)","LLA(52.2768, 21.0667, 0.0)",29448268,service
5,5,1495256494,8213623939,"LLA(52.2768, 21.0667, 0.0)","LLA(52.2757, 21.0658, 0.0)",29448268,service
6,6,1495256391,8213623939,"LLA(52.2757, 21.0657, 0.0)","LLA(52.2757, 21.0658, 0.0)",29448268,service
7,7,8213623939,1495256391,"LLA(52.2757, 21.0658, 0.0)","LLA(52.2757, 21.0657, 0.0)",29448268,service
8,8,10678694818,10678694819,"LLA(52.1685, 20.9307, 0.0)","LLA(52.1685, 20.9307, 0.0)",1147662891,service
9,9,10678694819,10678694818,"LLA(52.1685, 20.9307, 0.0)","LLA(52.1685, 20.9307, 0.0)",1147662891,service


In [14]:
dictionary, sparse_index = create_sparse_index(df.from,df.to,df.id,intersections)

(Dict(255152525 => 1, 4881584092 => 2, 5568665390 => 3, 9104166618 => 4, 11339122480 => 173817, 324549034 => 6, 11215827240 => 7, 10693459631 => 173818, 6832323964 => 9, 8893498647 => 10…), sparse([132948, 161221, 48035, 60970, 165382, 76819, 80535, 117941, 133389, 166510  …  119714, 26372, 71691, 21804, 124579, 34065, 166364, 46867, 80145, 103293], [1, 1, 2, 2, 2, 3, 3, 3, 4, 4  …  173814, 173815, 173815, 173816, 173816, 173817, 173817, 173818, 173818, 173818], [102651, 175502, 99473, 254971, 99472, 188901, 112890, 188900, 6017, 165528  …  102046, 226931, 156203, 121441, 178034, 237757, 49086, 20518, 23388, 20517], 173818, 173818))

In [15]:
edge = sparse_index[dictionary[4978125632],dictionary[1181144372]]

1